## Creating the game_logs for the basic statistics for every player based on retrosheet data.

I want to create a game_log sheet for every player in my data set. I realized this will help accelerate my backtesting. 

<br><br> The reason I am creating this file is that once I run the script once. I will just need to call the specific games to get the required information instead of generating it every time and not saving the results

<br> https://www.baseball-reference.com/players/gl.fcgi?id=bautijo02&t=b&year=2014  <br><br> this is the ideal format except the CSV file created will have all the players. So far it works from 2005-2018. File might be hundreds of MB but that is OK.


In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

d = pd.read_csv('../2005-2016_games.csv')
d.rename(columns={'unknown':'double_header_flag'}, inplace=True)
d['ab_flag'] = d.ab_flag.map({'F':0,'T':1})
d.sh_flag = d.sh_flag.map({'F':0,'T':1})
d.sf_flag = d.sf_flag.map({'F':0,'T':1})

In [2]:
df = d.copy()

In [3]:
#find me all the Plate Apperances as described by MLB rules
AB_only = df[df.ab_flag == 1]

event_flags = [14,15,16,17] #14 is BB 15 is IBB 16 is HBP 17 is interference
BB_IBB_HBP_INT = df[df.event_type.isin(event_flags)]

sac_hits = df[df.sh_flag ==1]

sac_fly  = df[df.sf_flag ==1]

#combine all the dataFrames into 1.
new_df = AB_only.append(BB_IBB_HBP_INT)
new_df = new_df.append(sac_hits)
new_df = new_df.append(sac_fly)

df = new_df.copy()

In [4]:
hit_flags = [20,21,22,23]     #20 = single, 21= double, 22 = triple, 23 = homerun
df.loc[df.event_type.isin(hit_flags), 'hit_flag'] = 1
df.loc[df.event_type.isin(hit_flags) == False, 'hit_flag'] = 0

#event_type 14 is BB, 15 is IBB, 16 is HBP, 17 is interference
df.loc[df.event_type==14, 'BB'] = 1
df.loc[df.event_type!=14, 'BB'] = 0

df.loc[df.event_type==15, 'IBB'] = 1
df.loc[df.event_type!=15, 'IBB'] = 0

df.loc[df.event_type==16, 'HBP'] = 1
df.loc[df.event_type!=16, 'HBP'] = 0

df.loc[df.event_type==17, 'ITF'] = 1
df.loc[df.event_type!=17, 'ITF'] = 0

df.loc[df.event_type==20, '1B'] = 1
df.loc[df.event_type!=20, '1B'] = 0

df.loc[df.event_type==21, '2B'] = 1
df.loc[df.event_type!=21, '2B'] = 0

df.loc[df.event_type==22, '3B'] = 1
df.loc[df.event_type!=22, '3B'] = 0

df.loc[df.event_type==23, 'HR'] = 1
df.loc[df.event_type!=23, 'HR'] = 0

In [5]:
data_grouped = df.groupby(['game_id','year','month','day','res_batter'])

#probably need to change game_id to double_header_flag that way but for now keep since you can just copy paste game_id on google and find the right game

In [6]:
# now that everything is seperated by game_id, 'year', 'month', 'day' and 'batter name' aggregate the stats
batting_summary = data_grouped.agg({'hit_flag':[np.sum, np.size],'ab_flag':[np.sum],'BB':np.sum,'IBB':np.sum,'HBP':np.sum,'ITF':np.sum,'1B':np.sum,'2B':np.sum,'3B':np.sum,'HR':np.sum,'sh_flag':np.sum,'sf_flag':np.sum})

In [7]:
#Just some cleaning
batting_hits_pa = batting_summary.hit_flag
batting_hits_pa.rename(columns={'sum':'H','size':'PA'}, inplace=True)


batting_ab = batting_summary.ab_flag
batting_ab.rename(columns={'sum':'AB'}, inplace=True)

batting_BB  = batting_summary.BB
batting_BB.rename(columns={'sum':'BB'}, inplace=True)

batting_IBB = batting_summary.IBB
batting_IBB.rename(columns={'sum':'IBB'}, inplace=True)

batting_HBP = batting_summary.HBP
batting_HBP.rename(columns={'sum':'HBP'}, inplace=True)

batting_ITF = batting_summary.ITF
batting_ITF.rename(columns={'sum':'ITF'}, inplace=True)

batting_1B = batting_summary['1B']
batting_1B.rename(columns={'sum':'1B'}, inplace=True)

batting_2B = batting_summary['2B']
batting_2B.rename(columns={'sum':'2B'}, inplace=True)

batting_3B = batting_summary['3B']
batting_3B.rename(columns={'sum':'3B'}, inplace=True)

batting_HR = batting_summary.HR
batting_HR.rename(columns={'sum':'HR'}, inplace=True)

batting_SH = batting_summary.sh_flag
batting_SH.rename(columns={'sum':'SH'}, inplace=True)

batting_SF = batting_summary.sf_flag
batting_SF.rename(columns={'sum':'SF'}, inplace=True)


#now combine all the smaller dataframes
batting_summary = pd.concat([batting_hits_pa,batting_ab,batting_BB,batting_IBB,batting_HBP,batting_ITF,batting_1B,batting_2B,batting_3B,batting_HR,batting_SH,batting_SF], axis =1)

batting_summary.sort_values(by=['res_batter','year','month','day','game_id'],inplace=True)

#calculate BA and OBP: BA = H / AB, OBP = (H + BB + IBB + HBP)/ (AB + BB + IBB + HBP + SF )
batting_summary['BA'] = batting_summary.groupby(['res_batter','year'])['H'].transform(pd.Series.cumsum) / batting_summary.groupby(['res_batter','year'])['AB'].transform(pd.Series.cumsum)
batting_summary['OBP'] = (batting_summary.groupby(['res_batter','year'])['H'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['BB'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['IBB'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['HBP'].transform(pd.Series.cumsum) ) / (batting_summary.groupby(['res_batter','year'])['AB'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['BB'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['IBB'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['HBP'].transform(pd.Series.cumsum) + batting_summary.groupby(['res_batter','year'])['SF'].transform(pd.Series.cumsum))

#calculate player's cumulative Hits to each day in the season, as well as his H/PA
batting_summary['H_cum_season'] = batting_summary.groupby(['res_batter','year'])['H'].transform(pd.Series.cumsum)
batting_summary['H/PA_seasonal'] =  batting_summary.groupby(['res_batter','year'])['H'].transform(pd.Series.cumsum) / batting_summary.groupby(['res_batter','year'])['PA'].transform(pd.Series.cumsum)

#for each game the player plays in a season... write down his game count. also calculate H/seasonal_game_played
batting_summary['seasonal_game_played']= batting_summary.groupby(['res_batter','year']).cumcount()+1
batting_summary['H/seasonal_game_played'] = batting_summary.groupby(['res_batter','year'])['H'].transform(pd.Series.cumsum) / batting_summary.seasonal_game_played

batting_summary.reset_index(inplace=True)

In [8]:
batting_summary.query('res_batter=="bautj002" and year == 2010').sample(5)

,game_id,year,month,day,res_batter,H,PA,AB,BB,IBB,...,3B,HR,SH,SF,BA,OBP,H_cum_season,H/PA_seasonal,seasonal_game_played,H/seasonal_game_played
31439,TOR201008260,2010,8,26,bautj002,2.0,4.0,3,1.0,0.0,...,0.0,1.0,0,0,0.260674,0.376404,116.0,0.217228,126,0.920635
31344,CHA201005070,2010,5,7,bautj002,0.0,6.0,4,2.0,0.0,...,0.0,0.0,0,0,0.215517,0.316176,25.0,0.183824,31,0.806452
31398,TOR201007080,2010,7,8,bautj002,1.0,4.0,4,0.0,0.0,...,0.0,1.0,0,0,0.238095,0.361582,70.0,0.197740,85,0.823529
31321,TOR201004130,2010,4,13,bautj002,0.0,4.0,3,1.0,0.0,...,0.0,0.0,0,0,0.161290,0.325000,5.0,0.125000,8,0.625000
31396,TOR201007060,2010,7,6,bautj002,0.0,4.0,4,0.0,0.0,...,0.0,0.0,0,0,0.232639,0.355491,67.0,0.193642,83,0.807229


In [9]:
# We got what we want, we can now save the dataframe
batting_summary.to_csv('../2005-2016_game_logs.csv',index=None)

## Running Tests

We can run tests on the code and verify the results by comparing them to the information on a reputable website: baseball-reference.com

<br> To do this we will pick a year, here I've chosen 2006 but this can easily be changed

<br> Next we will randomly pick a batter and visit his website and manually check the numbers

In [10]:
yearly_batting_summary = batting_summary.groupby('year')

In [11]:
df2006 = yearly_batting_summary.get_group(2006)

### Case 1: Checking Hits, AB, and PA

In [12]:
import random
#batter = random.sample(set(df2006.res_batter),1)
batter =['suzui001']
print(batter)

df2006[df2006.res_batter == batter[0]].H.sum(), df2006[df2006.res_batter == batter[0]].AB.sum(), df2006[df2006.res_batter == batter[0]].PA.sum()

['suzui001']


(224.0, 695, 752.0)

The chosen batter is 'suzui001' which is Ichiro Suzuki (https://www.retrosheet.org/boxesetc/S/Psuzui001.htm) <br><br> We can check his 2006 stats by following this link: https://www.baseball-reference.com/players/gl.fcgi?id=suzukic01&t=b&year=2006
<br><br>By scrolling to the bottom of the web-page, we can see that he had exactly 752 Plate appearances, 695 AB and 224 Hits which corresponds to the data we obtained

### Case 2: Checking BB, IBB and HBP,

In [13]:
df2006[df2006.res_batter == batter[0]].BB.sum(),df2006[df2006.res_batter == batter[0]].IBB.sum(),df2006[df2006.res_batter == batter[0]].HBP.sum()

(33.0, 16.0, 5.0)

Checking his Walks, Intentional Walks and Hit by Pitch on baseball-reference we can see that he has 51 Walks, with 1 intentional walks and 6 Hit by Pitches. <br><br>I've double checked the game date below and I can confirm that baseball-reference counts the Intentional walks in the Walks Columns which is why they have it at 51 (50 normal walks + 1 intentional walks) instead of 50

In [14]:
df2006.query('res_batter =="suzui001" and month==7 and day ==31')[['game_id','year','month','day','res_batter','BB','IBB']]

,game_id,year,month,day,res_batter,BB,IBB
539302,BAL200607310,2006,7,31,suzui001,0.0,1.0


In [15]:
df.query('year==2006 and res_batter =="suzui001" and month==7 and day ==31')

,game_id,away_team,inning,batting_team,outs,balls,strikes,vis_score,home_score,res_batter,...,double_header_flag,hit_flag,BB,IBB,HBP,ITF,1B,2B,3B,HR
241494,BAL200607310,SEA,1,0,0,3,2,0,0,suzui001,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241518,BAL200607310,SEA,4,0,0,2,0,0,3,suzui001,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241535,BAL200607310,SEA,6,0,1,2,1,0,3,suzui001,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241560,BAL200607310,SEA,8,0,1,1,1,8,4,suzui001,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241547,BAL200607310,SEA,7,0,1,3,0,4,3,suzui001,...,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Case 3: Checking his BA and OBP

In [16]:
df2006[df2006.res_batter==batter[0]].sample(3)[['game_id','year','month','day','BA','OBP']]

,game_id,year,month,day,BA,OBP
539299,CLE200607280,2006,7,28,0.347126,0.400000
539200,SEA200604050,2006,4,5,0.333333,0.466667
539230,SEA200605070,2006,5,7,0.270073,0.333333


https://www.baseball-reference.com/players/gl.fcgi?id=suzukic01&t=b&year=2006
<br><br>On July 17, 2006 Ichiro's BA was .340 and OBP was .395        : pass
<br>On  April 7, 2006: Ichiro's BA was .286 and OBP was .375 : pass
<br>On April 23, 2006: Ichiro's BA was .253 and OBP was .333      : pass